## Análise Exploratória, Descritiva, Econômica, Clusterização da produção do Cupuaçu no Amazonas de 2023 por Johann Kotaro

In [ ]:
# Bibliotecas necessárias para análise espacial
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
from geopy.geocoders import Nominatim
import time
from sklearn.cluster import KMeans


In [ ]:
# Carregar o arquivo Excel com os dados de produção de cupuaçu com coordenadas
df = pd.read_excel("producao_cupuaçu_com_coordenadas.xlsx")
df.head()  # Visualizar as primeiras linhas do DataFrame


,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.),Latitude,Longitude
0,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60,-3.364051,-68.199469
1,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00,-4.365855,-70.190874
2,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00,-4.378275,-70.029060
3,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20,-3.465063,-68.946846
4,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38,-4.232735,-69.926337


In [66]:
# Verificar se as coordenadas já foram obtidas corretamente
df[['Unidade local', 'Latitude', 'Longitude']].head()


,Unidade local,Latitude,Longitude
0,Amaturá,-3.364051,-68.199469
1,Atalaia do Norte,-4.365855,-70.190874
2,Benjamin Constant,-4.378275,-70.029060
3,São Paulo de Olivença,-3.465063,-68.946846
4,Tabatinga,-4.232735,-69.926337


In [ ]:
# Remover as linhas onde Latitude ou Longitude são NaN
gdf_clean = gdf.dropna(subset=['Latitude', 'Longitude'])

# Verificar se as linhas foram removidas
gdf_clean.head()


,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.),Latitude,Longitude,geometry
0,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60,-3.364051,-68.199469,POINT (-68.19947 -3.36405)
1,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00,-4.365855,-70.190874,POINT (-70.19087 -4.36585)
2,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00,-4.378275,-70.029060,POINT (-70.02906 -4.37827)
3,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20,-3.465063,-68.946846,POINT (-68.94685 -3.46506)
4,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38,-4.232735,-69.926337,POINT (-69.92634 -4.23273)


In [67]:
# Criar um GeoDataFrame com as coordenadas de latitude e longitude
gdf_clean = gpd.GeoDataFrame(gdf_clean, 
                             geometry=gpd.points_from_xy(gdf_clean['Longitude'], gdf_clean['Latitude']))

# Definir o CRS (Sistema de Referência de Coordenadas) adequado para o Amazonas (pode ser EPSG:4326)
gdf_clean.set_crs('EPSG:4326', inplace=True)

,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.),Latitude,Longitude,geometry
0,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60,-3.364051,-68.199469,POINT (-68.19947 -3.36405)
1,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00,-4.365855,-70.190874,POINT (-70.19087 -4.36585)
2,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00,-4.378275,-70.029060,POINT (-70.02906 -4.37827)
3,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20,-3.465063,-68.946846,POINT (-68.94685 -3.46506)
4,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38,-4.232735,-69.926337,POINT (-69.92634 -4.23273)
5,Alvarães,7,3.00,3.00,4.50,10,4.00,4.00,4.80,-3.215579,-64.814833,POINT (-64.81483 -3.21558)
6,Fonte Boa,7,1.30,1.30,2.60,8,1.50,1.50,2.60,-2.514746,-66.094842,POINT (-66.09484 -2.51475)
7,Japurá,9,2.60,0.60,0.60,28,9.60,7.60,6.08,-1.880847,-66.996766,POINT (-66.99677 -1.88085)
8,Juruá,31,5.94,5.94,11.88,32,8.00,8.00,12.80,-3.474667,-66.062234,POINT (-66.06223 -3.47467)
9,Jutaí,5,4.70,3.70,11.10,80,45.00,44.00,105.60,-2.747148,-66.772126,POINT (-66.77213 -2.74715)


In [68]:
# Criar o mapa interativo com as localizações limpas
m = folium.Map(location=[gdf_clean['Latitude'].mean(), gdf_clean['Longitude'].mean()], zoom_start=6)

# Adicionar os pontos ao mapa com um popup para cada unidade local
for _, row in gdf_clean.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Unidade local']).add_to(m)

# Exibir o mapa
m


In [69]:
import folium
from folium.plugins import HeatMap

# Criar um mapa base
m = folium.Map(location=[gdf_clean['Latitude'].mean(), gdf_clean['Longitude'].mean()], zoom_start=6)

# Adicionar uma camada de calor para visualizar a densidade de produção
heat_data = [[row['Latitude'], row['Longitude'], row['Produção (mil frutos)']] for _, row in gdf_clean.iterrows()]
HeatMap(heat_data).add_to(m)

# Exibir o mapa
m


In [ ]:
# Definir o número de clusters
kmeans = KMeans(n_clusters=4, random_state=42) # A quantidade de Clusters é definida arbitráriamente, ou seja, sem uso de otimização de quantidade de Clusters

# Aplicar o KMeans nas coordenadas (Latitude e Longitude)
coords = gdf_clean[['Latitude', 'Longitude']].values
gdf_clean['Cluster'] = kmeans.fit_predict(coords)


C:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [71]:
# Criar o mapa base
m = folium.Map(location=[gdf_clean['Latitude'].mean(), gdf_clean['Longitude'].mean()], zoom_start=6)

# Adicionar os pontos ao mapa, colorindo-os de acordo com o cluster
for _, row in gdf_clean.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], 
                        radius=5, 
                        color=f"#{hex(row['Cluster']*555555)[2:8]}",  # Cor diferente para cada cluster
                        fill=True, 
                        fill_color=f"#{hex(row['Cluster']*555555)[2:8]}").add_to(m)

# Exibir o mapa
m


In [ ]:
# Obter as coordenadas dos centros de cada cluster, os centróides
cluster_centers = kmeans.cluster_centers_

# Adicionar os centros no mapa
for center in cluster_centers:
    folium.Marker([center[0], center[1]], 
                  popup="Centro do Cluster", 
                  icon=folium.Icon(color='red')).add_to(m)

# Exibir o mapa com os centros dos clusters
m


In [ ]:
# Contar a quantidade de UL por cluster
cluster_counts = gdf_clean['Cluster'].value_counts()
print("Quantidade de Unidade Local pór Cluster")
print(cluster_counts)


2    24
1     9
3     9
0     9
Name: Cluster, dtype: int64


In [ ]:
# Exibir o GeoDataFrame completo
gdf_clean

,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.),Latitude,Longitude,geometry,Cluster
0,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60,-3.364051,-68.199469,POINT (-68.19947 -3.36405),1
1,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00,-4.365855,-70.190874,POINT (-70.19087 -4.36585),1
2,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00,-4.378275,-70.029060,POINT (-70.02906 -4.37827),1
3,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20,-3.465063,-68.946846,POINT (-68.94685 -3.46506),1
4,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38,-4.232735,-69.926337,POINT (-69.92634 -4.23273),1
5,Alvarães,7,3.00,3.00,4.50,10,4.00,4.00,4.80,-3.215579,-64.814833,POINT (-64.81483 -3.21558),3
6,Fonte Boa,7,1.30,1.30,2.60,8,1.50,1.50,2.60,-2.514746,-66.094842,POINT (-66.09484 -2.51475),3
7,Japurá,9,2.60,0.60,0.60,28,9.60,7.60,6.08,-1.880847,-66.996766,POINT (-66.99677 -1.88085),3
8,Juruá,31,5.94,5.94,11.88,32,8.00,8.00,12.80,-3.474667,-66.062234,POINT (-66.06223 -3.47467),3
9,Jutaí,5,4.70,3.70,11.10,80,45.00,44.00,105.60,-2.747148,-66.772126,POINT (-66.77213 -2.74715),3


In [75]:
gdf_clean.to_excel("resultado_kmeans_clusterizacao.xlsx", index=False)


## Análise Descritiva por Cluster

In [76]:
# Análise descritiva dos clusters
df_clusters = gdf_clean.groupby('Cluster').agg({
    'Latitude': ['mean', 'min', 'max'],
    'Longitude': ['mean', 'min', 'max'],
    'Plantada (ha)': 'mean',
    'Colhida (ha)': 'mean',
    'Produção (mil frutos)': 'mean',
    'Nº de Beneficiários': 'mean'
}).reset_index()

# Exibindo a análise descritiva
df_clusters


Cluster  Latitude                      Longitude                        \
               mean       min       max       mean        min        max   
0       0 -6.198116 -7.926952 -3.841320 -62.592484 -64.792451 -59.883019   
1       1 -5.265493 -7.437889 -3.364051 -69.680207 -71.691232 -68.199469   
2       2 -3.109046 -4.393478 -0.798652 -59.403568 -61.660520 -56.731932   
3       3 -2.077038 -3.474667 -0.122865 -65.609883 -67.096972 -62.925635   

  Plantada (ha) Colhida (ha) Produção (mil frutos) Nº de Beneficiários  
           mean         mean                  mean                mean  
0      5.845556     4.773333              9.618889            9.555556  
1      4.702222     4.546667              9.356667            9.333333  
2     26.492500    24.042500             52.431667           31.250000  
3      9.091111     5.553333             12.843333           25.222222

## Distribuição da Produção por Cluster

In [77]:
# Total de produção por cluster
total_producao_por_cluster = gdf_clean.groupby('Cluster')['Produção (mil frutos)'].sum()

# Porcentagem da produção total por cluster
percentual_producao = (total_producao_por_cluster / total_producao_por_cluster.sum()) * 100

# Exibindo os resultados
resultados_producao = pd.DataFrame({
    'Total de Produção (mil frutos)': total_producao_por_cluster,
    'Percentual da Produção (%)': percentual_producao
}).sort_values(by='Total de Produção (mil frutos)', ascending=False)

resultados_producao


,Total de Produção (mil frutos),Percentual da Produção (%)
Cluster,,
2,1258.36,81.461485
3,115.59,7.482861
0,86.57,5.604216
1,84.21,5.451438


## Análise de Comparação das Áreas Plantadas e Colhidas

In [78]:
# Comparando as áreas plantadas e colhidas por cluster
df_areas_comparativas = gdf_clean.groupby('Cluster').agg({
    'Plantada (ha)': 'sum',
    'Colhida (ha)': 'sum'
}).reset_index()

# Exibindo as comparações
df_areas_comparativas['Taxa de Colheita (%)'] = (df_areas_comparativas['Colhida (ha)'] / df_areas_comparativas['Plantada (ha)']) * 100

df_areas_comparativas


,Cluster,Plantada (ha),Colhida (ha),Taxa de Colheita (%)
0,0,52.61,42.96,81.657480
1,1,42.32,40.92,96.691871
2,2,635.82,577.02,90.752100
3,3,81.82,49.98,61.085309


## Análise de Eficiência (Produção por Hectare)

In [79]:
# Calculando a produção por hectare por cluster
gdf_clean['Producao por Hectare'] = gdf_clean['Produção (mil frutos)'] / gdf_clean['Plantada (ha)']

# Média de produção por hectare por cluster
df_producao_eficiencia = gdf_clean.groupby('Cluster')['Producao por Hectare'].mean()

df_producao_eficiencia


Cluster
0    1.711470
1    1.787778
2    2.052504
3    1.325028
Name: Producao por Hectare, dtype: float64

## Análise de Correlacionamento Entre Variáveis

In [80]:
# Analisando correlação entre as variáveis numéricas
correlacao = gdf_clean[['Plantada (ha)', 'Colhida (ha)', 'Produção (mil frutos)', 'Nº de Beneficiários']].corr()

# Exibindo a matriz de correlação
correlacao


,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários
Plantada (ha),1.000000,0.988708,0.986023,0.904132
Colhida (ha),0.988708,1.000000,0.995178,0.846489
Produção (mil frutos),0.986023,0.995178,1.000000,0.847596
Nº de Beneficiários,0.904132,0.846489,0.847596,1.000000


## Comparação de Beneficiários Entre os Clusters

In [81]:
# Número médio de beneficiários por cluster
beneficiarios_por_cluster = gdf_clean.groupby('Cluster')['Nº de Beneficiários'].mean()

# Exibindo o número médio de beneficiários
beneficiarios_por_cluster


Cluster
0     9.555556
1     9.333333
2    31.250000
3    25.222222
Name: Nº de Beneficiários, dtype: float64

## Tabela com Unidade local por Cluster

In [82]:
# Agrupar as unidades locais por cluster e reclassificar o índice
tabela_clusters = gdf_clean[['Unidade local', 'Cluster']].sort_values(by='Cluster').reset_index(drop=True)

# Exibindo a tabela com as Unidades Locais por Cluster e índice reclassificado
tabela_clusters


,Unidade local,Cluster
0,Codajás,0
1,Santo Antônio do Matupi,0
2,Manicoré,0
3,Humaitá,0
4,Apuí,0
5,Coari,0
6,Tapauá,0
7,Lábrea,0
8,Canutama,0
9,Itamarati,1


In [83]:
# Salvando a tabela como um arquivo Excel
tabela_clusters.to_excel("unidades_locais_por_cluster.xlsx", index=False)
